In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/CNNWMA/data/logs"

In [ ]:
 # change to your saved location
%cd /content/drive/MyDrive/TL_WMA/data 

In [ ]:
# FOLDER COUNT INCREMENT
exp_sv_dir=''
def inc_dir(fldr,occ):
  runs=fldr
  sv_dir= runs+'/exp_'+str(len(os.listdir(runs))+occ)
  if(not os.path.exists(sv_dir)):
    os.mkdir(sv_dir)
    global exp_sv_dir
    exp_sv_dir = sv_dir
    return sv_dir
  else:
    print("path exist")
    inc_dir(runs,occ+1)

#SETUP

import os,tqdm
import pandas as pd
import cv2
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import numpy as np

# CALL BACKS
from keras.callbacks import ModelCheckpoint, EarlyStopping ,ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard


clsnm= ['Burn','Hematoma','Abscess','Abrasion','Cut/Incision']

x_test=np.load("x_test.npy")
x_train=np.load("x_train.npy")
x_val=np.load("x_val.npy")
y_train=np.load("y_train.npy")
y_test=np.load("y_test.npy")
y_val=np.load("y_val.npy")

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print(x_val.shape,y_val.shape)

#HOT ONE ENCODE
from sklearn.utils.multiclass import unique_labels
from tensorflow.keras.utils  import to_categorical

y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)


print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

In [ ]:
mdl_noaug = Sequential()
mdl_noaug.add(Conv2D(32, (3, 3), input_shape=(227,227,3)))
mdl_noaug.add(Activation('relu'))
mdl_noaug.add(MaxPooling2D(pool_size=(3, 3)))
mdl_noaug.add(BatchNormalization())

mdl_noaug.add(Conv2D(32, (3, 3)))
mdl_noaug.add(Activation('relu'))
mdl_noaug.add(MaxPooling2D(pool_size=(5, 5)))
mdl_noaug.add(BatchNormalization())

mdl_noaug.add(Conv2D(32, (3, 3)))
mdl_noaug.add(Activation('relu'))
mdl_noaug.add(MaxPooling2D(pool_size=(5, 5)))
mdl_noaug.add(BatchNormalization())

mdl_noaug.add(Flatten()) 

mdl_noaug.add(Dense(32))
mdl_noaug.add(Activation('relu'))
mdl_noaug.add(Dropout(0.6))

mdl_noaug.add(Dense(5))
mdl_noaug.add(Activation('softmax'))

mdl_noaug.summary()


In [ ]:
#Training the model #NO AUG

mdl_noaug.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

tb_callback = TensorBoard(log_dir=inc_dir(r"/content/drive/MyDrive/CNNWMA/data/logs",0))
ml_best= exp_sv_dir + '/cjbest.h5'

mc = ModelCheckpoint(ml_best, monitor='val_loss', mode='min', verbose=1, save_best_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)
lrr= ReduceLROnPlateau( monitor='val_loss', factor=.1,  patience=5, min_lr=0.00001)

epochs=300
batch_size = 64
%pwd
mdl_noaug.fit(x_train, y_train, epochs = epochs,
            validation_data = (x_val,y_val), 
            callbacks = [tb_callback,es,mc,lrr],verbose=1)

print(exp_sv_dir)
model_dir = exp_sv_dir+"/cj_model.h5"
mdl_noaug.save(model_dir)
his_dir = exp_sv_dir +'/cjnet_history.npy'
np.save(his_dir,mdl_noaug.history)

In [ ]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from keras.models import load_model
 
#GRAPHS & TESTING

#num=9
#svd_pth = r"/content/drive/MyDrive/CNNWMA/data/logs" #seeting directory
#svd_model= svd_pth+'/exp_'+str(num)+'/cjbest.h5' #setting file path

C7NET = load_model("cjbest.h5")
print(svd_model)


#svd_his= svd_pth+'/exp_'+str(num) +'/cjnet_history.npy'
C7his=np.load("cjnet_history.npy",allow_pickle='TRUE').item()
#After successful training, we will visualize its performance.
#print(svd_his)


predict_x=C7NET.predict(x_test) 
y_pred=np.argmax(predict_x,axis=1)
y_true=np.argmax(y_test,axis=1)

acc_score = accuracy_score(y_true, y_pred)
print('Accuracy Score = ', acc_score)



#Plotting the training and validation loss

f,ax=plt.subplots(1,2) #Creates 2 subplots under 1 column
f.set_figwidth(17)
f.set_figheight(8)

#Assigning the first subplot to graph training loss and validation loss
ax[0].plot(C7his.history['loss'][20:],color='b',label='Training Loss')
ax[0].plot(C7his.history['val_loss'][20:],color='r',label='Validation Loss')
ax[0].legend()

#Plotting the training accuracy and validation accuracy
ax[1].plot(C7his.history['accuracy'],color='b',label='Training  Accuracy')
ax[1].plot(C7his.history['val_accuracy'],color='r',label='Validation Accuracy')
ax[1].legend()

plt.show()

C7NET.summary()

In [ ]:
#CONFUSION MATRIX
#Defining function for confusion matrix plot
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#Print Confusion matrix
    fig, ax = plt.subplots(figsize=(7,7))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

np.set_printoptions(precision=2)
#Plotting the confusion matrix
from sklearn.metrics import confusion_matrix
#confusion_mtx=confusion_matrix(y_true,y_pred)

clsnm= ['Burn','Hematoma','Abscess','Abrasion','Cut/Incision']

# Plotting non-normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes = clsnm,title = 'Confusion matrix, Normalized',normalize=True)